In [35]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
nii_object = nib.load('Data/I004_3.nii.gz')
nii_data = nii_object.get_fdata()

# print(nii_object)
print(nii_data)

# def show_slices(slices):
#     fig, axes = plt.subplots(1,len(slices))
#     for i, slice in enumerate(slices):
#         axes[i].imshow(slice.T, cmap="gray", origin="lower")

# slice_0 = nii_data[int(nii_data.shape[0]/2), :, :]
# slice_1 = nii_data[:,int(nii_data.shape[1]/2), :]
# slice_2 = nii_data[:, :, int(nii_data.shape[2]/2)]
# slice_3 = nii_data[:,:,3]
# show_slices([slice_0, slice_1, slice_2,slice_3])
# plt.suptitle("Center slices for EPI image")




ImportError: /home/jp/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: cusparseSpSM_analysis, version libcusparse.so.11

In [3]:
import os
import numpy as np
import nibabel as nib
from scipy import ndimage
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def read(path):
    scan = nib.load(path)
    scan = scan.get_fdata()
    return scan

# normalizing the volume between the ranges of [-1000,1000]
def normalize(volume):
    min = -1000
    max = 1000
    volume[volume < min] = min
    volume[volume < max] = max
    volume = volume.astype("float32")
    return volume 

# to lower computational cost, resize the 3D image across the z axis
def resize(img):
    
    setDepth = 64
    setWidth = 128
    setHeight = 128

    currentDepth = img.shape[2]
    currentWidth = img.shape[1]
    currentHeight = img.shape[0]

    d = currentDepth / setDepth
    w = currentWidth / setWidth
    h  = currentHeight / setHeight

    depthFactor = 1 / d 
    widthFactor = 1 / w 
    heightFactor = 1 / h
    
    img = ndimage.rotate(img, 90, reshape=False)
    img  = ndimage.zoom(img, (widthFactor, heightFactor, depthFactor))

    return img


def process_img(path):
    volume = read(path)
    volume = normalize(volume)
    volume = resize(volume)
    return volume 



2023-03-27 16:32:12.177373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 16:32:12.332819: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-27 16:32:12.336316: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 16:32:12.336332: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [ ]:
# Creating np.arrays for the 3D images and their respective ground truth

#Creating the paths for these images
non_target_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/0", x)
    for x in os.listdir("labeled_data/data/0")
]
saline_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/1", x)
    for x in os.listdir("labeled_data/data/1")
]
rubber_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/2", x)
    for x in os.listdir("labeled_data/data/2")
]
clay_paths = [
    os.path.join(os.getcwd(), "labeled_data/data/3", x)
    for x in os.listdir("labeled_data/data/3")
]

print(len(non_target_paths))
print(len(saline_paths))
print(len(rubber_paths))
print(len(clay_paths))

# Creating np.array for respective images feature

non_target_imgs = np.array([process_img(path) for path in non_target_paths])
saline_imgs = np.array([process_img(path) for path in saline_paths])
rubber_imgs = np.array([process_img(path) for path in rubber_paths])
clay_imgs = np.array([process_img(path) for path in clay_paths])

# now we are assigning their respective ground truth

non_target = np.array([0 for _ in range(len(non_target_imgs))])
saline_target = np.array([1 for _ in range(len(saline_imgs))])
rubber_target = np.array([2 for _ in range(len(rubber_imgs))])
clay_target = np.array([3 for _ in range(len(clay_imgs))])



In [22]:
print(os.getcwd())

data_folder = f'{os.getcwd()}/labeled_data/data/'

X = []
Y = []
for label in os.listdir(data_folder):
    img_directory = f'{data_folder}/{label}/'
    for img in os.listdir(img_directory):
        X += [process_img(f'{img_directory}/{img}')]
        Y += [label]
        
        
X = np.array(X, dtype='object')
Y = np.array(Y)


/home/jp/Documents/GitHub/ATR_Project


In [34]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

test1 = np.asarray(x_train)
test2 = np.asarray(y_train)
test1 = tf.convert_to_tensor(x_train)
# train_loader = tf.data.Dataset.from_tensor_slices(test1, test2)
# validation_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).